In [1]:
using Revise

using Test
import TensorCrossInterpolation as TCI
import Random

In [2]:
using Test
import TensorCrossInterpolation as TCI
import TensorCrossInterpolation: rank, linkdims, TensorCI2, MultiIndex, evaluate, crossinterpolate2, pivoterror, tensortrain
import Random
import QuanticsGrids as QD

In [3]:
import TensorCrossInterpolation as TCI
using TensorCrossInterpolation
using Random
using Test
using ITensors
ITensors.disable_warn_order()

14

In [4]:
pivotsearch = :full
#seed = 12456
#seed = 1245678
seed = 124567890

Random.seed!(seed)

R = 10
abstol = 1e-2
sites = [Index(2, "n=$n") for n in 1:R]

Ψ = random_mps(sites; linkdims=20)
rindex = Set([rand(1:2, R) for _ in 1:20])

tensor = Array(reduce(*, Ψ), reverse(sites))
tensor = tensor ./ maximum(abs, tensor)

function f(x)
    res = tensor[x...]
    if x in rindex
        res += 1.01 * abstol
    end
    res
end

localdims = fill(2, R)
firstpivot = ones(Int, R)
tci, ranks, errors = crossinterpolate2(
    Float64,
    f,
    localdims,
    [firstpivot];
    tolerance=abstol,
    maxbonddim=1000,
    maxiter=20,
    loginterval=1,
    verbosity=1,
    normalizeerror=false,
    pivotsearch=pivotsearch,
)
@show abstol

iteration = 1, rank = 10, error= 0.0, maxsamplevalue= 0.8233588094296722, nglobalpivot=5
  Rejected 5 global pivots added in the previous iteration, errors are [2.220446049250313e-16, 1.6653345369377348e-16, 4.440892098500626e-16, 0.0, 1.1102230246251565e-16]
iteration = 2, rank = 26, error= 0.006575621006188141, maxsamplevalue= 1.0, nglobalpivot=0
iteration = 3, rank = 26, error= 0.006575621006188141, maxsamplevalue= 1.0, nglobalpivot=0
iteration = 4, rank = 26, error= 0.006575621006188141, maxsamplevalue= 1.0, nglobalpivot=0
abstol = 0.01


0.01

In [5]:
TCI.addglobalpivots2sitesweep!(
    tci, f, collect(rindex),
    tolerance=abstol,
    normalizeerror=false,
    maxbonddim=typemax(Int),
    pivotsearch=pivotsearch,
    verbosity=1,
)


Trying to add 19 global pivots, 0 still remain.


0

In [7]:
tt = TCI.TensorTrain(tci)
@test sum(abs.([TCI.evaluate(tt, r) - f(r) for r in rindex]) .> abstol) == 0

Test Passed

In [12]:
tt = TCI.TensorTrain(tci)
TCI.compress!(tt, :SVD; tolerance=1e-10)

@show TCI.linkdims(tt)

println("Error on pivots:")
for b in 1:length(tci)-1
    Iset = tci.Iset[b+1]
    Jset = tci.Jset[b]
    diff = maximum(abs, [TCI.evaluate(tt, vcat(i, j)) - f(vcat(i, j)) for i in Iset, j in Jset])
    println(b, "    ", diff, " ", diff > abstol)
end
println("Error on T:")
for l in 1:length(tci)-1
    diff = [TCI.evaluate(tt, vcat(i, m, j)) - f(vcat(i, m, j)) for i in tci.Iset[l], j in tci.Jset[l], m in 1:localdims[l]]
    println(l, "    ", maximum(abs, diff), " ", maximum(abs, diff) > abstol)
end

TCI.linkdims(tt) = [2, 4, 8, 16, 26, 16, 8, 4, 2]
Error on pivots:
1    1.2323475573339238e-14 false
2    1.2323475573339238e-14 false
3    1.2323475573339238e-14 false
4    1.2323475573339238e-14 false
5    1.4765966227514582e-14 false
6    1.4765966227514582e-14 false
7    1.2323475573339238e-14 false
8    1.2323475573339238e-14 false
9    4.440892098500626e-15 false
Error on T:
1    1.2323475573339238e-14 false
2    1.2323475573339238e-14 false
3    1.2323475573339238e-14 false
4    1.2323475573339238e-14 false
5    1.4765966227514582e-14 false
6    1.4765966227514582e-14 false
7    1.4765966227514582e-14 false
8    1.2323475573339238e-14 false
9    1.2323475573339238e-14 false
